<a href="https://colab.research.google.com/github/daniahamuda/flower-sketch-classifier/blob/main/traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="snA4vsNSH6S3ia5q9egX")
project = rf.workspace("traffic-tmfdb").project("vechiles-hq6la-r7fco")
version = project.version(2)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import os

images_dir = "/content/Vechiles-2/train/images"
labels_dir = "/content/Vechiles-2/train/labels"

image_exts = (".jpg", ".jpeg", ".png")

images = sorted([
    os.path.splitext(f)[0]
    for f in os.listdir(images_dir)
    if f.lower().endswith(image_exts)
])

labels = sorted([
    os.path.splitext(f)[0]
    for f in os.listdir(labels_dir)
    if f.lower().endswith(".txt")
])

print("Total images:", len(images))
print("Total labels:", len(labels))

image_set = set(images)
label_set = set(labels)

images_without_labels = image_set - label_set
labels_without_images = label_set - image_set

if not images_without_labels and not labels_without_images:
    print("\n✅ RESULT: Every image has a matching label and vice versa")
else:
    print("\n❌ RESULT: Mismatches found")

    if images_without_labels:
        print("\nImages without labels:")
        for f in sorted(images_without_labels)[:10]:
            print(" ", f)

    if labels_without_images:
        print("\nLabels without images:")
        for f in sorted(labels_without_images)[:10]:
            print(" ", f)


In [ ]:
import os
from PIL import Image


In [ ]:
images_dir = "/content/Vechiles-2/train/images"  # change if needed


In [ ]:
converted = 0
skipped = 0

for filename in os.listdir(images_dir):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(images_dir, filename)

        try:
            with Image.open(img_path) as img:
                # Check image mode
                if img.mode != "RGB":
                    img = img.convert("RGB")
                    img.save(img_path)
                    converted += 1
                else:
                    skipped += 1

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

print(f"✅ Converted to RGB: {converted}")
print(f"⏭️ Already RGB (skipped): {skipped}")


In [ ]:
import os
from PIL import Image, ImageOps


In [ ]:
fixed = 0
skipped = 0

for filename in os.listdir(images_dir):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(images_dir, filename)

        try:
            with Image.open(img_path) as img:
                # Apply EXIF-based auto orientation
                img_oriented = ImageOps.exif_transpose(img)

                # Save back only if something changed
                if img_oriented != img:
                    img_oriented.save(img_path)
                    fixed += 1
                else:
                    skipped += 1

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

print(f"🔄 Auto-oriented images: {fixed}")
print(f"⏭️ No orientation change needed: {skipped}")


In [ ]:
import os
import cv2
import numpy as np
from collections import Counter


In [ ]:
base_path = "/content/Vechiles-2/train"
images_dir = os.path.join(base_path, "images")
labels_dir = os.path.join(base_path, "labels")


In [ ]:
image_sizes = []

for img_name in os.listdir(images_dir):
    if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
        img = cv2.imread(os.path.join(images_dir, img_name))
        h, w, _ = img.shape
        image_sizes.append((w, h))

widths, heights = zip(*image_sizes)

print("=== IMAGE SIZE ANALYSIS ===")
print(f"Total images: {len(image_sizes)}")
print(f"Min size: {min(widths)}x{min(heights)}")
print(f"Max size: {max(widths)}x{max(heights)}")
print(f"Avg size: {int(np.mean(widths))}x{int(np.mean(heights))}")


In [ ]:
empty_labels = []

for label_name in os.listdir(labels_dir):
    label_path = os.path.join(labels_dir, label_name)
    if os.path.getsize(label_path) == 0:
        empty_labels.append(label_name)

print("\n=== EMPTY LABELS ===")
print(f"Empty label files: {len(empty_labels)}")


In [ ]:
class_counter = Counter()

for label_name in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_name)) as f:
        for line in f:
            class_id = int(line.split()[0])
            class_counter[class_id] += 1

print("\n=== CLASS DISTRIBUTION ===")
for cls, count in class_counter.items():
    print(f"Class {cls}: {count}")


In [ ]:
box_areas = []
small_boxes = 0

for label_name in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_name)) as f:
        for line in f:
            _, _, _, w, h = map(float, line.split())
            area = w * h
            box_areas.append(area)

            if area < 0.01:  # <1% of image
                small_boxes += 1

print("\n=== BOUNDING BOX ANALYSIS ===")
print(f"Total boxes: {len(box_areas)}")
print(f"Avg box area (%): {np.mean(box_areas)*100:.2f}")
print(f"Small boxes (<1%): {small_boxes}")


In [ ]:
aspect_ratios = []

for label_name in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_name)) as f:
        for line in f:
            _, _, _, w, h = map(float, line.split())
            if h > 0:
                aspect_ratios.append(w / h)

print("\n=== ASPECT RATIO ===")
print(f"Avg aspect ratio: {np.mean(aspect_ratios):.2f}")
